# Trabalho Prático 2 - Planejamento e Navegação
## Passo 2b: Informed RRT* para Kuka youbot

### Importação de Bibliotecas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from shapely.geometry import Point, LineString, Polygon
from descartes.patch import PolygonPatch

### Definição dos Parâmetros

O Informed RRT* é uma extensão do RRT* que utiliza heurísticas para focar a amostragem
em regiões do espaço que podem melhorar a solução atual. Após encontrar uma solução inicial,
o algoritmo restringe a amostragem a um subconjunto elipsoidal do espaço de configuração.

In [ ]:
# Dimensões do mundo
WORLD_WIDTH = 50
WORLD_HEIGHT = 30

# Parâmetros do Kuka youbot
ROBOT_RADIUS = 0.2
SAFETY_MARGIN = 0.1
EFFECTIVE_RADIUS = ROBOT_RADIUS + SAFETY_MARGIN

# Parâmetros do Informed RRT*
MAX_ITERATIONS = 2000
STEP_SIZE = 1.0
GOAL_SAMPLE_RATE = 0.05  # 5% de chance de amostrar o objetivo
SEARCH_RADIUS = 3.0  # raio de busca para rewiring
GOAL_THRESHOLD = 0.5  # distância para considerar que chegou ao objetivo

### Carregamento dos Mapas

In [ ]:
# Carregar mapas
mapa1 = mpimg.imread('imagens/mapa1_invertido.png')
mapa2 = mpimg.imread('imagens/mapa2_invertido.png')

### Implementação do Informed RRT*

O algoritmo Informed RRT* funciona em duas fases:

1. **Fase Inicial (RRT*)**: Constrói a árvore até encontrar uma solução inicial
2. **Fase Informada**: Restringe a amostragem a uma elipse definida pelos focos no início e objetivo,
   com eixo maior igual ao custo da melhor solução atual

A elipse é definida por:
- Focos: $x_{start}$ e $x_{goal}$
- Eixo maior: $c_{best}$ (custo da melhor solução)
- Eixo menor: $\sqrt{c_{best}^2 - c_{min}^2}$, onde $c_{min}$ é a distância euclidiana entre início e objetivo

In [ ]:
class Node:
    """Nó da árvore RRT*."""
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent = None
        self.cost = 0.0
    
    def position(self):
        return (self.x, self.y)

def distance(node1, node2):
    """Calcula distância euclidiana entre dois nós."""
    return np.sqrt((node1.x - node2.x)**2 + (node1.y - node2.y)**2)

def world_to_pixel(x, y, img_shape, world_width, world_height):
    """Converte coordenadas do mundo para coordenadas de pixel."""
    h, w = img_shape[:2]
    px = int((x / world_width) * w)
    py = int((y / world_height) * h)
    return px, py

def is_collision_free(node1, node2, mapa, robot_radius, world_width, world_height):
    """Verifica se o caminho entre dois nós está livre de colisões."""
    num_checks = int(distance(node1, node2) * 10)
    num_checks = max(num_checks, 10)
    
    for i in range(num_checks + 1):
        t = i / num_checks
        x = node1.x + t * (node2.x - node1.x)
        y = node1.y + t * (node2.y - node1.y)
        
        px, py = world_to_pixel(x, y, mapa.shape, world_width, world_height)
        
        # Verificar área ao redor do ponto
        radius_px = int((robot_radius / world_width) * mapa.shape[1])
        
        for dx in range(-radius_px, radius_px + 1):
            for dy in range(-radius_px, radius_px + 1):
                if dx*dx + dy*dy <= radius_px*radius_px:
                    check_px = px + dx
                    check_py = py + dy
                    
                    if 0 <= check_px < mapa.shape[1] and 0 <= check_py < mapa.shape[0]:
                        if mapa[check_py, check_px, 0] < 0.5:
                            return False
    return True

def sample_ellipsoid(c_best, c_min, x_center, C):
    """Amostra um ponto dentro do elipsoide informado.
    
    Baseado em: Gammell et al., "Informed RRT*: Optimal Sampling-based 
    Path Planning Focused via Direct Sampling of an Admissible Ellipsoidal Heuristic"
    """
    if c_best < float('inf'):
        # Raios da elipse
        r1 = c_best / 2.0
        r_i = np.sqrt(c_best**2 - c_min**2) / 2.0
        
        # Amostra em esfera unitária
        theta = random.uniform(0, 2*np.pi)
        r = random.uniform(0, 1)
        
        # Transformar para elipse
        x_ball = np.array([r * np.cos(theta), r * np.sin(theta)])
        
        # Escalar pelos raios
        L = np.diag([r1, r_i])
        
        # Transformar para o espaço do mundo
        x_rand = C @ L @ x_ball + x_center
        
        return x_rand[0], x_rand[1]
    else:
        # Amostragem uniforme
        return random.uniform(0, WORLD_WIDTH), random.uniform(0, WORLD_HEIGHT)

def get_rotation_to_world_frame(x_start, x_goal):
    """Calcula a matriz de rotação do frame da elipse para o frame do mundo."""
    a1 = np.array([x_goal[0] - x_start[0], x_goal[1] - x_start[1]])
    e1 = a1 / np.linalg.norm(a1)
    
    # Vetor perpendicular
    e2 = np.array([-e1[1], e1[0]])
    
    C = np.column_stack([e1, e2])
    return C

def nearest_node(nodes, sample):
    """Encontra o nó mais próximo da amostra."""
    min_dist = float('inf')
    nearest = None
    
    for node in nodes:
        d = np.sqrt((node.x - sample[0])**2 + (node.y - sample[1])**2)
        if d < min_dist:
            min_dist = d
            nearest = node
    
    return nearest

def steer(from_node, to_point, step_size):
    """Cria um novo nó na direção do ponto alvo."""
    direction = np.array([to_point[0] - from_node.x, to_point[1] - from_node.y])
    dist = np.linalg.norm(direction)
    
    if dist < step_size:
        return Node(to_point[0], to_point[1])
    
    direction = direction / dist
    new_x = from_node.x + step_size * direction[0]
    new_y = from_node.y + step_size * direction[1]
    
    return Node(new_x, new_y)

def near_nodes(nodes, node, radius):
    """Encontra nós dentro de um raio."""
    near = []
    for n in nodes:
        if distance(n, node) < radius:
            near.append(n)
    return near

def choose_parent(near_nodes_list, nearest, new_node, mapa, robot_radius, world_width, world_height):
    """Escolhe o melhor pai para o novo nó."""
    if not near_nodes_list:
        return nearest
    
    min_cost = nearest.cost + distance(nearest, new_node)
    best_parent = nearest
    
    for node in near_nodes_list:
        if is_collision_free(node, new_node, mapa, robot_radius, world_width, world_height):
            cost = node.cost + distance(node, new_node)
            if cost < min_cost:
                min_cost = cost
                best_parent = node
    
    return best_parent

def rewire(nodes, new_node, near_nodes_list, mapa, robot_radius, world_width, world_height):
    """Reconecta nós próximos se o novo nó oferece um caminho melhor."""
    for node in near_nodes_list:
        if node == new_node.parent:
            continue
        
        if is_collision_free(new_node, node, mapa, robot_radius, world_width, world_height):
            new_cost = new_node.cost + distance(new_node, node)
            if new_cost < node.cost:
                node.parent = new_node
                node.cost = new_cost

def extract_path(goal_node):
    """Extrai o caminho do nó objetivo até a raiz."""
    path = []
    node = goal_node
    
    while node is not None:
        path.append(node.position())
        node = node.parent
    
    return list(reversed(path))

def informed_rrt_star(start, goal, mapa, max_iterations, step_size, goal_sample_rate,
                      search_radius, goal_threshold, robot_radius, world_width, world_height):
    """Implementa o algoritmo Informed RRT*."""
    # Inicializar árvore
    start_node = Node(start[0], start[1])
    nodes = [start_node]
    
    # Melhor solução encontrada
    best_goal_node = None
    c_best = float('inf')
    
    # Distância mínima (heurística)
    c_min = np.sqrt((goal[0] - start[0])**2 + (goal[1] - start[1])**2)
    
    # Centro da elipse
    x_center = np.array([(start[0] + goal[0])/2, (start[1] + goal[1])/2])
    
    # Matriz de rotação
    C = get_rotation_to_world_frame(start, goal)
    
    solution_found_iteration = None
    
    for i in range(max_iterations):
        # Amostragem
        if random.random() < goal_sample_rate:
            sample = goal
        else:
            if best_goal_node is not None:
                # Amostragem informada
                sample = sample_ellipsoid(c_best, c_min, x_center, C)
            else:
                # Amostragem uniforme
                sample = (random.uniform(0, world_width), random.uniform(0, world_height))
        
        # Encontrar nó mais próximo
        nearest = nearest_node(nodes, sample)
        
        # Criar novo nó
        new_node = steer(nearest, sample, step_size)
        
        # Verificar colisão
        if not is_collision_free(nearest, new_node, mapa, robot_radius, world_width, world_height):
            continue
        
        # Encontrar nós próximos
        near = near_nodes(nodes, new_node, search_radius)
        
        # Escolher melhor pai
        best_parent = choose_parent(near, nearest, new_node, mapa, robot_radius, world_width, world_height)
        new_node.parent = best_parent
        new_node.cost = best_parent.cost + distance(best_parent, new_node)
        
        # Adicionar à árvore
        nodes.append(new_node)
        
        # Rewiring
        rewire(nodes, new_node, near, mapa, robot_radius, world_width, world_height)
        
        # Verificar se chegou ao objetivo
        dist_to_goal = np.sqrt((new_node.x - goal[0])**2 + (new_node.y - goal[1])**2)
        if dist_to_goal < goal_threshold:
            if new_node.cost < c_best:
                best_goal_node = new_node
                c_best = new_node.cost
                if solution_found_iteration is None:
                    solution_found_iteration = i
                    print(f"Solução inicial encontrada na iteração {i} com custo {c_best:.2f}")
    
    if best_goal_node is not None:
        path = extract_path(best_goal_node)
        print(f"Solução final: custo {c_best:.2f}")
        return path, nodes, c_best
    else:
        print("Nenhuma solução encontrada")
        return None, nodes, float('inf')

### Experimento 1: Mapa 1 - Cenário Simples

In [ ]:
# Definir pontos de início e objetivo
start_mapa1 = (5, 15)
goal_mapa1 = (45, 15)

# Executar Informed RRT*
path_mapa1, nodes_mapa1, cost_mapa1 = informed_rrt_star(
    start_mapa1, goal_mapa1, mapa1, MAX_ITERATIONS, STEP_SIZE, GOAL_SAMPLE_RATE,
    SEARCH_RADIUS, GOAL_THRESHOLD, EFFECTIVE_RADIUS, WORLD_WIDTH, WORLD_HEIGHT
)

if path_mapa1:
    print(f"Número de nós na árvore: {len(nodes_mapa1)}")
    print(f"Número de waypoints no caminho: {len(path_mapa1)}")

### Visualização do Resultado - Mapa 1

In [ ]:
if path_mapa1:
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(mapa1, extent=[0, WORLD_WIDTH, 0, WORLD_HEIGHT], origin='lower')
    
    # Desenhar árvore
    for node in nodes_mapa1:
        if node.parent:
            ax.plot([node.x, node.parent.x], [node.y, node.parent.y], 
                   'b-', alpha=0.2, linewidth=0.5)
    
    # Desenhar caminho
    path_x = [p[0] for p in path_mapa1]
    path_y = [p[1] for p in path_mapa1]
    ax.plot(path_x, path_y, 'r-', linewidth=3)
    
    # Desenhar início e objetivo
    ax.plot(start_mapa1[0], start_mapa1[1], 'go', markersize=10)
    ax.plot(goal_mapa1[0], goal_mapa1[1], 'ro', markersize=10)
    
    ax.set_xlim(0, WORLD_WIDTH)
    ax.set_ylim(0, WORLD_HEIGHT)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig('imagens/informed_rrt_star_mapa1_youbot.png', dpi=150, bbox_inches='tight')
    plt.show()

### Experimento 2: Mapa 2 - Cenário Complexo

In [ ]:
# Definir pontos de início e objetivo
start_mapa2 = (5, 5)
goal_mapa2 = (45, 25)

# Executar Informed RRT*
path_mapa2, nodes_mapa2, cost_mapa2 = informed_rrt_star(
    start_mapa2, goal_mapa2, mapa2, MAX_ITERATIONS, STEP_SIZE, GOAL_SAMPLE_RATE,
    SEARCH_RADIUS, GOAL_THRESHOLD, EFFECTIVE_RADIUS, WORLD_WIDTH, WORLD_HEIGHT
)

if path_mapa2:
    print(f"Número de nós na árvore: {len(nodes_mapa2)}")
    print(f"Número de waypoints no caminho: {len(path_mapa2)}")

### Visualização do Resultado - Mapa 2

In [ ]:
if path_mapa2:
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(mapa2, extent=[0, WORLD_WIDTH, 0, WORLD_HEIGHT], origin='lower')
    
    # Desenhar árvore
    for node in nodes_mapa2:
        if node.parent:
            ax.plot([node.x, node.parent.x], [node.y, node.parent.y], 
                   'b-', alpha=0.2, linewidth=0.5)
    
    # Desenhar caminho
    path_x = [p[0] for p in path_mapa2]
    path_y = [p[1] for p in path_mapa2]
    ax.plot(path_x, path_y, 'r-', linewidth=3)
    
    # Desenhar início e objetivo
    ax.plot(start_mapa2[0], start_mapa2[1], 'go', markersize=10)
    ax.plot(goal_mapa2[0], goal_mapa2[1], 'ro', markersize=10)
    
    ax.set_xlim(0, WORLD_WIDTH)
    ax.set_ylim(0, WORLD_HEIGHT)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig('imagens/informed_rrt_star_mapa2_youbot.png', dpi=150, bbox_inches='tight')
    plt.show()

### Análise dos Resultados

O algoritmo Informed RRT* apresenta as seguintes características:

1. **Completude Probabilística**: Garante encontrar uma solução se ela existir, dado tempo suficiente
2. **Otimalidade Assintótica**: Converge para a solução ótima conforme o número de iterações aumenta
3. **Amostragem Informada**: Após encontrar uma solução inicial, restringe a amostragem a um subconjunto elipsoidal,
   focando em regiões que podem melhorar a solução
4. **Eficiência**: Mais eficiente que RRT* padrão em termos de convergência para o ótimo
5. **Rewiring**: Reconecta nós para melhorar continuamente a qualidade da solução

**Referência Principal:**
Gammell, J. D., Srinivasa, S. S., & Barfoot, T. D. (2014). 
"Informed RRT*: Optimal Sampling-based Path Planning Focused via Direct Sampling of an Admissible Ellipsoidal Heuristic."
IEEE/RSJ International Conference on Intelligent Robots and Systems (IROS).

O método é particularmente adequado para o Kuka youbot devido à sua capacidade de gerar trajetórias
de alta qualidade que podem ser seguidas diretamente pelo robô holonômico.